# LAB 3: Speech-to-text with RNN



![alt text](https://miro.medium.com/max/556/1*NhOH4X9wKWfO6o8faYFf-w.png)

# Introduction

In this lab, we are going to use Recurrent Neural Networks to do some Speech Recognition.

Nowadays, speech recognition is a common task present in smart home assistants (Amazon Echo, Google Home), phones, TVs... Most of the time, it is done using deep learning.

## What you will learn

- The different kinds of RNN (RNN, LSTM, GRU...)

- How to load and process audio data in PyTorch

- How to implement an RNN in PyTorch

- How to create a confusion matrix


## RNN

Recurrent Neural Networks are a kind of Neural Network used to process **sequences** of data.
These sequences can be of varying length and usually have some context information.

For example, sentences (text), audio, videos have some temporal logic. In a sentence, one word depends on the word that comes before it. In a video, one frame probably looks a lot like the previous frames.

RNNs have some kind of **persistence** of information during the processing of a sequence. Thus, RNNs are used for lots of things: sentiment analysis, text completion, speech recognition, etc.

![alt text](https://www.researchgate.net/profile/Weijiang_Feng/publication/318332317/figure/fig1/AS:614309562437664@1523474221928/The-standard-RNN-and-unfolded-RNN.png)

## LSTM (Long Short-Term Memory) and GRU (Gated Recurrent Unit)

**RNN Short-term memory problem:**

*   Recurrent Neural Networks suffer from short-term memory. If a sequence is long enough, they’ll have a hard time carrying information from earlier time steps to later ones. So if you are trying to process a paragraph of text to do predictions, RNN’s may leave out important information from the beginning.

*  During back propagation, recurrent neural networks suffer from the vanishing gradient problem. Gradients are values used to update a neural networks weights. The vanishing gradient problem is when the gradient shrinks as it back propagates through time. If a gradient value becomes extremely small, it doesn’t contribute too much to the learning.

**As solution to short-term memory, LSTM and GRU were created:**

*   LSTM was introduced by this [article](https://www.bioinf.jku.at/publications/older/2604.pdf).
*   GRU was introduced by this [article](https://arxiv.org/pdf/1412.3555.pdf).

Both are **Reccurent Neural Network (RNN)** architectures which were created as the solution to short-term memory. They have internal mechanisms called gates that can regulate the flow of information.

![alt text](http://dprogrammer.org/wp-content/uploads/2019/04/RNN-vs-LSTM-vs-GRU-1200x361.png)









These gates can learn which data in a sequence is important to keep or throw away. By doing that, it can pass relevant information down the long chain of sequences to make predictions.

**LSTM**

The LSTMs does have the ability to remove or add information to the cell state, carefully regulated by structures called gates.

*   the **cell state** make easy for the information to pass through the cell by combining the cells decisions
*   the **forget gate** decides what information should be thrown away from the cell state
*   the **input gate** decides which values we'll update using sigmoid ; it's combined with a tanh layer to create an update to the state
*   the **output gate**, based on the celle state, output a filtered information

**GRU**

The GRU is a modified version of the LSTM. It combines the forget and input gates into a single “update gate.” It also merges the cell state and hidden state, and makes some other changes. The resulting model is simpler than standard LSTM models, and has been growing increasingly popular.



## Libraries

Since we are working with PyTorch and sounds, we are going to use *torchaudio* instead of *torchvision*, this time.

Make sure you are using a GPU Runtime! (Runtime -> Change Runtime type)

In [ ]:
!pip install torchaudio==0.9.0

In [ ]:
from IPython.display import Audio

## PyTorch things
import torch
import torchaudio
import torch.nn.functional as F

## Other libs
import matplotlib.pyplot as plt
import glob
import os
import random
from tqdm import tqdm_notebook
import torchsummary
import numpy as np
from sklearn.metrics import confusion_matrix
from sklearn.preprocessing import normalize
import pandas as pd
import seaborn as sn

# Part 1: Working with audio data

The dataset we are using is Google's Speech Dataset (https://ai.googleblog.com/2017/08/launching-speech-commands-dataset.html).

It is composed of **"65,000 one-second long utterances of 30 short words, by thousands of different people"**.

Let's download the dataset:

In [ ]:
!rm -rf ./*
!wget -O speech_commands_v0.01.tar.gz http://download.tensorflow.org/data/speech_commands_v0.01.tar.gz
!tar xzf speech_commands_v0.01.tar.gz
!ls

Let's print the different classes (words) that are part of this dataset.

We can see there are 30 different words.

In [ ]:
classes = os.listdir()
classes.remove("LICENSE")
classes.remove("README.md")
classes.remove("_background_noise_")
classes.remove("speech_commands_v0.01.tar.gz")
classes.remove("testing_list.txt")
classes.remove("validation_list.txt")
classes.remove(".config")
print(classes)
print("Number of classes", len(classes))

## Q1: Listen to some samples

Using the **Audio(filename)** function from IPython notebook, you can listen to an audio file directly in Colab.

Try it on some samples!

In [ ]:
Audio("bed/1528225c_nohash_2.wav")

In [ ]:
Audio(## Try another file here)

## Q2: Displaying a waveform

Use **torchaudio.load** to load an audio file. Then, use matplotlib to display it.

HINT: you may have to transpose the waveform with **.t()** in order to display it

In [ ]:
## YOUR CODE HERE

## Computing MFCC features

Extracting MFCC (**Mel Frequency Cepstral Coefficients**) features is a well known signal processing technique, especially used in **ASR** (Automatic Speech Recognition). These features are meant to represent the way humans perceive sound. https://en.wikipedia.org/wiki/Mel-frequency_cepstrum

*Torchaudio* has transforms (just like the ones in *torchvision*) that allow you to compute these features:

In [ ]:
mfcc = torchaudio.transforms.MFCC(n_mfcc=12, log_mels=True)(waveform)
plt.figure()
plt.imshow(mfcc[0].detach().numpy())
print(mfcc.shape)

Here, we are only keeping 12 MFCC features because it is enough for our purposes.

As you can see, we are getting a Tensor of shape [1, 12, 81], because we have one audio channel (mono) with 12 coefficients over 81 time windows.

## Creating a custom audio Dataset

You may have noticed that in this dataset, the test and validation datasets are given in testing_list.txt and validation_list.txt files.

With that, we can infer a training list as well:

In [ ]:
## Read the test list
with open("testing_list.txt") as testing_f:
  testing_list = [x.strip() for x in testing_f.readlines()]

## Read the val list
with open("validation_list.txt") as val_f:
  validation_list = [x.strip() for x in val_f.readlines()]

print("Number of testing samples", len(testing_list))
print("Number of validation samples", len(validation_list))

## Construct a train list
training_list = []
for c in classes:
  training_list += glob.glob(c + "/*")

training_list = list(filter(lambda x : not x in testing_list and not x in validation_list, training_list))
print("Number of training samples", len(training_list))

Now, we can create a custom SpeechDataset class that takes a file list in input.

In [ ]:
class SpeechDataset(torch.utils.data.Dataset):

  def __init__(self, classes, file_list):

    self.classes = classes

    # create a map from class name to integer
    self.class_to_int = dict(zip(classes, range(len(classes))))

    # store the file names
    self.samples = file_list

    # store our MFCC transform
    self.mfcc_transform = torchaudio.transforms.MFCC(n_mfcc=12, log_mels=True)

  def __len__(self):
    return len(self.samples)

  def __getitem__(self,i):
    with torch.no_grad():
      # load a normalized waveform
      waveform,_ = torchaudio.load(self.samples[i], normalize=True)

      # if the waveform is too short (less than 1 second) we pad it with zeroes
      if waveform.shape[1] < 16000:
        waveform = F.pad(input=waveform, pad=(0, 16000 - waveform.shape[1]), mode='constant', value=0)

      # then, we apply the transform
      mfcc = self.mfcc_transform(waveform).squeeze(0).transpose(0,1)

    # get the label from the file name
    label = self.samples[i].split("/")[0]

    # return the mfcc coefficient with the sample label
    return mfcc, self.class_to_int[label]

## Q3: Create instances of the SpeechDataset for the train and val sets

Fill the code below to create your Dataset objects for training.

In [ ]:
train_set = ##YOUR CODE HERE
val_set = ##YOUR CODE HERE

## Q4: Create Dataloaders for training and validation

Fill the code below to create DataLoaders with the Datasets you just created.

Do not forget to add shuffling to the training DataLoader.

Print a batch of data to make sure everything works.

In [ ]:
train_dl = ##YOUR CODE HERE
val_dl = ##YOUR CODE HERE

# Part 2: Implementing a simple Recurrent Neural Network

For our network, we are going to use an **RNN module** from torch.nn (which can have multiple layers, or cells).

This module has an **input size**, which in our case will be equal to **the number of MFCC features (12)**. The input size is the number of dimensions of **x** in the image below.

It also has an **hidden size**, which is the size of the output of the layers as well as the size of the internal representation of the features. We are going to choose **256** to start, but feel free to change that. This is the dimension of **h** in the image below.

PyTorch RNN modules have a **number of layers**, which is simply the number of stacked **RNN Cells**. We are going to use 2 cells here, but feel free to change that as well. This is the **depth** in the image below.

Then, in order to get as many output as the number of classes in our dataset, we need to have a **Linear layer** that goes from **256 inputs (the hidden size) to 30 outputs (the number of classes).**

Finally, to output categorical probabilities, we use a **Softmax layer.**

![alt text](https://i.stack.imgur.com/SjnTl.png)

## Q4: Implement the network

Fill the code below to implement the network.

In [ ]:
class SpeechRNN(torch.nn.Module):

  def __init__(self):
    super(SpeechRNN, self).__init__()

    self.lstm = torch.nn.RNN(##YOUR CODE HERE

    self.out_layer = ##YOUR CODE HERE

    self.softmax = torch.nn.LogSoftmax(dim=1)

  def forward(self, x):

    out, _ = self.lstm(x)

    x = self.out_layer(out[:,-1,:])

    return self.softmax(x)

Use this code to check that your implementation is working.

In [ ]:
net = SpeechRNN()
batch = next(iter(train_dl))[0]
print(batch.shape)
y = net(batch)

print(y.shape)

# Part 3: Training the network

As usual, we need to define a loss and an optimizer. Since we have a categorical classification problem, we use cross-entropy (negative log likelihood).

We can use the Adam optimizer, feel free to change it or the learning rate.

In [ ]:
##RE-RUN THIS CODE TO GET A "NEW" NETWORK

LEARNING_RATE = 0.001

## Create an instance of our network
net = SpeechRNN()

## Move it to the GPU
net = net.cuda()

# Negative log likelihood loss
criterion = torch.nn.NLLLoss()

# Adam optimizer
optimizer = torch.optim.Adam(net.parameters(), lr=LEARNING_RATE)

## Q5: Training loop

We also need to write a training loop. Fill the code below to create it:

In [ ]:
## NUMBER OF EPOCHS TO TRAIN
N_EPOCHS = 5

epoch_loss, epoch_acc, epoch_val_loss, epoch_val_acc = [], [], [], []

for e in range(N_EPOCHS):

  print("EPOCH:",e)

  ### TRAINING LOOP
  running_loss = 0
  running_accuracy = 0

  ## Put the network in training mode
  net.train()

  for i, batch in enumerate(tqdm_notebook(train_dl)):

  ##YOUR CODE HERE

    ## Compute some statistics
    with torch.no_grad():
      running_loss += loss.item()
      running_accuracy += (y.max(1)[1] == labels).sum().item()

  print("Training accuracy:", running_accuracy/float(len(train_set)),
        "Training loss:", running_loss/float(len(train_set)))

  epoch_loss.append(running_loss/len(train_set))
  epoch_acc.append(running_accuracy/len(train_set))

  ### VALIDATION LOOP
  ## Put the network in validation mode
  net.eval()

  running_val_loss = 0
  running_val_accuracy = 0

  for i, batch in enumerate(val_dl):

    with torch.no_grad():
       #YOUR CODE HERE

      running_val_loss += loss.item()
      running_val_accuracy += (y.max(1)[1] == labels).sum().item()

  print("Validation accuracy:", running_val_accuracy/float(len(val_set)),
        "Validation loss:", running_val_loss/float(len(val_set)))

  epoch_val_loss.append(running_val_loss/len(val_set))
  epoch_val_acc.append(running_val_accuracy/len(val_set))


## Q6: From RNN to LSTM/GRU

As you can see, the accuracy is pretty bad when we are only using "regular" RNNs. These are not used very much in practice nowadays because they do not have long-term memory. This means that by the time the network is done processing the whole audio sample, it probably has already forgotten the important parts of it. **Replace the RNN module in your network (Q4) with an LSTM or a GRU module (as you want). Train a new network and watch that accuracy go up!**

# Part 4: Evaluation

Now, we need to evaluate our network on the test set.

Use the code below to do that:

In [ ]:
# Create a test dataset instance
test_dataset = SpeechDataset(classes, testing_list)

# Create a DataLoader
test_dl = torch.utils.data.DataLoader(test_dataset, batch_size=64)

net.eval()

test_loss = 0
test_accuracy = 0

preds, y_test = np.array([]), np.array([])

for i, batch in enumerate(test_dl):

  with torch.no_grad():
    # Get a batch from the dataloader
    x = batch[0]
    labels = batch[1]

    # move the batch to GPU
    x = x.cuda()
    labels = labels.cuda()

    # Compute the network output
    y = net(x)

    # Compute the loss
    loss = criterion(y, labels)

    ## Store all the predictions an labels for later
    preds = np.hstack([preds, y.max(1)[1].cpu().numpy()])
    y_test = np.hstack([y_test, labels.cpu().numpy()])

    test_loss += loss.item()
    test_accuracy += (y.max(1)[1] == labels).sum().item()

print("Test accuracy:", test_accuracy/float(len(test_dataset)),
      "Test loss:", test_loss/float(len(test_dataset)))

## Confusion matrix

In classification problems, it is common to use a confusion matrix to visualize which particular classes the model struggles with.

Use the code below to generate a confusion matrix. What is the confusion that your network makes the most?

In [ ]:
def show_confusion_matrix(pred, Y_TEST, classes):

  cm = confusion_matrix(y_true=Y_TEST, y_pred=pred)
  cm = normalize(cm,axis=1,norm='l1')

  df_cm = pd.DataFrame(cm, index = classes, columns = classes)

  plt.figure(figsize=(30,15))

  cmap = sn.cubehelix_palette(light=1, as_cmap=True)
  sn.heatmap(df_cm, annot=True,cmap=cmap)
  plt.title('Confusion Matrix',fontdict={'fontsize':20})
  plt.xlabel('Predicted labels')
  plt.ylabel('True labels')
  plt.show


show_confusion_matrix(preds, y_test, classes)

# (OPTIONAL) Part 5: Going further

## Kaggle audio recognition datasets

If you want to do more audio classification, there have been two competitions on Kaggle:

- https://www.kaggle.com/c/freesound-audio-tagging

- https://www.kaggle.com/c/freesound-audio-tagging-2019

![alt text](https://storage.googleapis.com/kaggle-media/competitions/freesound/task2_freesound_audio_tagging.png)

## More RNN things

There are lots of things you can do with RNN. Natural Language Processing (NLP) is really popular right now.
You can follow PyTorch tutorials to get started!

https://becominghuman.ai/a-simple-introduction-to-natural-language-processing-ea66a1747b32

https://pytorch.org/tutorials/beginner/deep_learning_nlp_tutorial.html